## Using Selenium

To use selenium, we need to create an instance that is going to "drive" us through the webpage

In [1]:
from selenium import webdriver
from time import sleep

driver = webdriver.Chrome()
driver.get("https://zoopla.co.uk")

Cool! We see that we've navigated to the Zoopla.co.uk website. We can search for elements via `xpath` and can also send mouse and keyboard actions through Selenium as well. Let's recall the challenge we want to solve - extracting data for 50 houses:
- **Sale Price**: Our response variable
- Number of bedrooms
- Square footage
- Description
- Address

We'll focus our efforts just in the London area the next cell will take us to the URL corresponding to properties in London

In [2]:
driver = webdriver.Chrome() 
URL = "https://www.zoopla.co.uk/new-homes/property/london/?q=London&results_sort=newest_listings&search_source=new-homes&page_size=25&pn=1&view_type=list"
driver.get(URL)

Oh... Looks like cookies are blocking us... We need to find a way to get around this 🤔. Let's start by using xpath to find the "Accept All Cookies" button

UPDATE: The zoopla website has a frame in the website. The 'Accept Cookies' is in this frame, so we have to tell selenium to access the frame. Usually, if it doesn't have a frame, you can ignore the `switch_to_frame` method

In [7]:
from selenium import webdriver
import time

driver = webdriver.Chrome() 
URL = "https://www.zoopla.co.uk/new-homes/property/london/?q=London&results_sort=newest_listings&search_source=new-homes&page_size=25&pn=1&view_type=list"
driver.get(URL)
time.sleep(2) # Wait a couple of seconds, so the website doesn't suspect you are a bot
try:
    driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame
    accept_cookies_button = driver.find_element_by_xpath('//*[@id="save"]')
    accept_cookies_button.click()

except AttributeError: # If you have the latest version of selenium, the code above won't run because the "switch_to_frame" is deprecated
    driver.switch_to.frame('gdpr-consent-notice') # This is the id of the frame
    accept_cookies_button = driver.find_element_by_xpath('//*[@id="save"]')
    accept_cookies_button.click()

except:
    pass # If there is no cookies button, we won't find it, so we can pass

<ipython-input-7-b34e9bb16874>:9: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame


Wow, have you seen that? The webdriver went to the website and it clicked the button for us! So, analyze the methods we used:
- `find_element_by_xpath()` To make the driver point to the element 
- `click()` To make the driver click on the element that was pointed

Alright, so it is time to start extracting the data we are interested on. Let's extract the price, address, number of bedrooms and the description

First of all, observe the HTML code corresponding to a property:
<p align=center><img src=images/Selenium_1.png width=900></p>

If you get the XPath of that property, it will look like this:

`//*[@id="listing_60212639"]`

Which is fine if we want to find a single property, but not so great if we want to list all the properties in that page. We will focus on how to get all the properties shortly, for now, let's extract the URL of that property, and extract the information we need. 

__IMPORTANT! Zoopa is constantly adding new properties, it is likely that the Xpath changed, so make sure that you are following all the steps and using the correct XPath__

Let's take a look again at the HTML code, you will notice that there are some `<a>` tags in the HTML code. Usually, these tags are used to include a hyper reference (`href`). Selenium allows us to get that href, but first we need to locate the `<a>` tag containing the href.

So, if you expand one of the `<div>` tags corresponding to a property, you will see something like this:

<p align=center><img src=images/Selenium_2.png width=900></p>

Can you see the `<a>` tag? That is the tag that contains the URL we need. So, let's tell selenium to extract it

In [12]:
from selenium import webdriver
import time

driver = webdriver.Chrome() 
URL = "https://www.zoopla.co.uk/new-homes/property/london/?q=London&results_sort=newest_listings&search_source=new-homes&page_size=25&pn=1&view_type=list"
driver.get(URL)
time.sleep(2) # Wait a couple of seconds, so the website doesn't suspect you are a bot
try:
    driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame
    accept_cookies_button = driver.find_element_by_xpath('//*[@id="save"]')
    accept_cookies_button.click()

except AttributeError: # If you have the latest version of selenium, the code above won't run because the "switch_to_frame" is deprecated
    driver.switch_to.frame('gdpr-consent-notice') # This is the id of the frame
    accept_cookies_button = driver.find_element_by_xpath('//*[@id="save"]')
    accept_cookies_button.click()

except:
    pass
time.sleep(2)
property = driver.find_element_by_xpath('//*[@id="listing_60212639"]') # Change this xpath with the xpath the current page has in their properties
a_tag = property.find_element_by_tag_name('a')
link = a_tag.get_attribute('href')
print(link)


<ipython-input-12-08ba0b8dd794>:9: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame


https://www.zoopla.co.uk/new-homes/details/60212639/


Nice, now we can visit that link using selenium. Alternatively, you can also click on the `property` element (`property.click()`) and it will take you to the same page. But you will have to:
- Click the element
- Sleep
- Extract the information
- Go back
- Sleep
- Find the next property 
- Click
- Sleep

On the other hand, if you have the links, you can visit them like this:

- Extract all the links
- Iterate through the list, and for each iteration, visit the corresponding URL
- Sleep
- Extract the information of the property
- Visit the next URL

So, it's up to you, but for many different websites, creating a list with links (which is usually called "crawler"), is much more efficient

Enough talking (or writing), let's visit the link we extracted

In [13]:
driver.get(link)

And it moved us to the webpage of that property

<p align=center><img src=images/Selenium_3.png width=900></p>

There, you can see the price, address, number of bedrooms, and the description. As always, let's take a look at the XPath corresponding to each property

<p align=center><img src=images/Selenium_4.png width=900></p>

And there it is, if you do the same with the number of bedrooms, the address and the desciption, you should have something like the following.

In [14]:
price = driver.find_element_by_xpath('//span[@data-testid="price"]').text
print(price)
address = driver.find_element_by_xpath('//span[@data-testid="address-label"]').text
print(address)
bedrooms = driver.find_element_by_xpath('//span[@data-testid="beds-label"]').text
print(bedrooms)
div_tag = driver.find_element_by_xpath('//div[@data-testid="truncated_text_container"]')
span_tag = div_tag.find_element_by_xpath('.//span')
description = span_tag.text
print(description)

£7,750,000
West Heath Avenue, Hampstead, London NW11
6 beds
A striking contemporary home with an abundance of light and space.

Location

The property is ideally situated for public transport and the national road network. Golders Green Underground Station (Northern Line) and Bus Terminus is just 400 metres walk, whilst there is easy road access to Brent Cross Shopping Centre, the A406 North Circular Road, the A41/A1 arterial route and junction 1 of the M1 Motorway.

Golders Hill Park is within 160 metres walk and offers beautiful plant displays, enhancing the peaceful setting of the Mediterranean and water gardens, while the park houses a popular café. There are also a variety of leisure facilities including tennis courts, croquet lawn, golf practice nets, butterfly house and a children's play area. In the park is also a free zoo, with a growing collection of rare and exotic birds and mammals such as laughing kookaburras, ring-tailed lemurs and ring-tailed coatis.



A striking home s

Now that we have a button, we can send a click action to it!

In [18]:
dict_properties = {'Price': [], 'Address': [], 'Bedrooms': [], 'Description': []}
price = driver.find_element_by_xpath('//span[@data-testid="price"]').text
dict_properties['Price'].append(price)
address = driver.find_element_by_xpath('//span[@data-testid="address-label"]').text
dict_properties['Address'].append(address)
bedrooms = driver.find_element_by_xpath('//span[@data-testid="beds-label"]').text
dict_properties['Bedrooms'].append(bedrooms)
div_tag = driver.find_element_by_xpath('//div[@data-testid="truncated_text_container"]')
span_tag = div_tag.find_element_by_xpath('.//span')
description = span_tag.text
dict_properties['Description'] = description

In [19]:
dict_properties

{'Price': ['£7,750,000'],
 'Address': ['West Heath Avenue, Hampstead, London NW11'],
 'Bedrooms': ['6 beds'],
 'Description': "A striking contemporary home with an abundance of light and space.\n\nLocation\n\nThe property is ideally situated for public transport and the national road network. Golders Green Underground Station (Northern Line) and Bus Terminus is just 400 metres walk, whilst there is easy road access to Brent Cross Shopping Centre, the A406 North Circular Road, the A41/A1 arterial route and junction 1 of the M1 Motorway.\n\nGolders Hill Park is within 160 metres walk and offers beautiful plant displays, enhancing the peaceful setting of the Mediterranean and water gardens, while the park houses a popular café. There are also a variety of leisure facilities including tennis courts, croquet lawn, golf practice nets, butterfly house and a children's play area. In the park is also a free zoo, with a growing collection of rare and exotic birds and mammals such as laughing koo

## Adding links to a list: Creating a Crawler

As mentioned, it would be more efficient to create a list with all the links and then iterate through that list. Here, I am going to give a small teaser of what it looks like, but, ultimately, it will be your task to complete the whole scraper

Before we move on, I am going to create a list with the accept cookies functionality, so we don't have to repeat myself so many times

In [43]:
from selenium import webdriver
import time

def load_and_accept_cookies() -> webdriver.Chrome:
    '''
    Open Zoopla and accept the cookies
    
    Returns
    -------
    driver: webdriver.Chrome
        This driver is already in the Zoopla webpage
    '''
    driver = webdriver.Chrome() 
    URL = "https://www.zoopla.co.uk/new-homes/property/london/?q=London&results_sort=newest_listings&search_source=new-homes&page_size=25&pn=1&view_type=list"
    driver.get(URL)
    time.sleep(3) 
    try:
        driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame
        accept_cookies_button = driver.find_element_by_xpath('//*[@id="save"]')
        accept_cookies_button.click()
        time.sleep(1)
    except AttributeError: # If you have the latest version of selenium, the code above won't run because the "switch_to_frame" is deprecated
        driver.switch_to.frame('gdpr-consent-notice') # This is the id of the frame
        accept_cookies_button = driver.find_element_by_xpath('//*[@id="save"]')
        accept_cookies_button.click()
        time.sleep(1)

    except:
        pass

    return driver 

Awesome, let's use this function from now on, it will make our code much more readable

In [25]:
driver = load_and_accept_cookies() # In case it works, driver should be in the Zoopla webpage with the cookies button clicked

<ipython-input-24-502712965682>:18: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame


Great, let's observe the list of properties one more time. All the properties are in a container as we can see in this image:

<p align=center><img src=images/Selenium_5.png width=900></p>

And, each property is one `<div>` tag inside that container. For example, for the two first properties:

<p align=center><img src=images/Selenium_7.png width=450><img src=images/Selenium_6.png width=450></p>

So, we can find a way to iterate through ALL the properties in that list, and for each iteration, extract the link. We saw earlier that if you have the property, you can easily find the `<a>` tag that contains the `href` like this:
```
property = driver.find_element_by_xpath('//*[@id="listing_60212639"]') # Change this xpath with the xpath the current page has in their properties
a_tag = property.find_element_by_tag_name('a')
link = a_tag.get_attribute('href')
```
Let's use something similar

In [32]:
driver = load_and_accept_cookies()
prop_container = driver.find_element_by_xpath('//*[@class="css-1anhqz4-ListingsContainer earci3d2"]') # XPath corresponding to the Container

<ipython-input-24-502712965682>:18: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame


Now, `prop_container` is pointing to the list of properties in the website. We have to get ALL the `<div>` tags inside, but only those that are direct children. So, we have to use a relative xpath: `./div`
- The dot represents that it is relative
- The single slash represents direct children

Also, take into account that we are looking for ALL occurence of this XPath, so we have to use the `find_elementS_by_xpath` method


In [41]:
prop_list = prop_container.find_elements_by_xpath('./div')
link_list = []

for property in prop_list:
    a_tag = property.find_element_by_tag_name('a')
    link = a_tag.get_attribute('href')
    link_list.append(link)
    
print(f'There are {len(link_list)} properties in this page')
print(link_list)

There are 25 properties in this page
['https://www.zoopla.co.uk/new-homes/details/60214186/', 'https://www.zoopla.co.uk/new-homes/details/60214076/', 'https://www.zoopla.co.uk/new-homes/details/60213997/', 'https://www.zoopla.co.uk/new-homes/details/60213955/', 'https://www.zoopla.co.uk/new-homes/details/60213931/', 'https://www.zoopla.co.uk/new-homes/details/60213891/', 'https://www.zoopla.co.uk/new-homes/details/60213807/', 'https://www.zoopla.co.uk/new-homes/details/60213785/', 'https://www.zoopla.co.uk/new-homes/details/60213797/', 'https://www.zoopla.co.uk/new-homes/details/60213732/', 'https://www.zoopla.co.uk/new-homes/details/60213753/', 'https://www.zoopla.co.uk/new-homes/details/60213127/', 'https://www.zoopla.co.uk/new-homes/details/60213083/', 'https://www.zoopla.co.uk/new-homes/details/60213079/', 'https://www.zoopla.co.uk/new-homes/details/60212733/', 'https://www.zoopla.co.uk/new-homes/details/59458212/', 'https://www.zoopla.co.uk/new-homes/details/60212693/', 'https://w

Now we have a list of the links of the properties in that page. How awesome is that?

Next, we need to iterate through this list and start visiting each link to extract the data we were interested on (Price, Address, Number of Bedroom, Description)

## Try it out

This will finish the Zoopla practical that you have in the Portal. 

With the new acquired knowledge, extract the data from all the properties in 5 different Zoopla pages. This means that, once you finish scraping a page, you have to click the 'Next Page' button (you can also change the URL if you know how to tweak it). So, once you extract the 25 links, you can go to the next page by clicking 'Next':

<p align=center><img src=images/Selenium_8.png width=450></p>

I included a template you can use to get started:

In [ ]:
from selenium import webdriver

def get_links(driver: webdriver.Chrome) -> list:
    '''
    Returns a list with all the links in the current page
    Parameters
    ----------
    driver: webdriver.Chrome
        The driver that contains information about the current page
    
    Returns
    -------
    link_list: list
        A list with all the links in the page
    '''

    prop_container = driver.find_element_by_xpath('//*[@class="css-1anhqz4-ListingsContainer earci3d2"]')
    prop_list = prop_container.find_elements_by_xpath('./div')
    link_list = []

    for property in prop_list:
        a_tag = property.find_element_by_tag_name('a')
        link = a_tag.get_attribute('href')
        link_list.append(link)

    return link_list

big_list = []
driver = load_and_accept_cookies()

for i in range(5): # The first 5 pages only
    big_list.extend(get_links(driver)) # Call the function we just created and extend the big list with the returned list
    ## TODO: Click the next button. Don't forget to use sleeps, so the website doesn't suspect
    pass # This pass should be removed once the code is complete


for link in big_list:
    ## TODO: Visit all the links, and extract the data. Don't forget to use sleeps, so the website doesn't suspect
    pass # This pass should be removed once the code is complete

driver.quit() # Close the browser when you finish

# Key Takeaways

